In [5]:
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np
from cmlreaders import CMLReader, get_data_index
from ptsa.data.filters import ButterworthFilter
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import *
from copy import copy
from scipy import stats
import pickle
plt.rcParams['pdf.fonttype'] = 42; plt.rcParams['ps.fonttype'] = 42 # fix fonts for Illustrator
sys.path.append('/home1/john/Downloads/ptsa_plot-master/')
sys.path.append('/home1/john/johnModules')
from brain_labels import MTL_labels, LTC_labels, PFC_labels, OTHER_labels, ALL_labels # all location labels
%load_ext autoreload
%autoreload
from general import *
from SWRmodule import *

# usually within local run
import mne
from scipy.signal import firwin,filtfilt,kaiserord
import pingouin as pg
%autoreload
HPC_labels,ENT_labels,PHC_labels = getMTLregions(MTL_labels)

# specific to loading UTSW data
import warnings
warnings.simplefilter('ignore')
from ptsa.data.readers import EEGReader
from ptsa.data.MatlabIO import read_single_matlab_matrix_as_numpy_structured_array
from glob import glob


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
### PARAMS ###

save_values = 0

selected_period = 'surrounding_recall' 
# 'surrounding_recall': aligned to time of free recall 
# 'whole_retrieval': aligned to beginning of retrieval period (beep_off)
# 'encoding': aligned to word_on 
# 'math': aligned to math problem on
# 'math_retrieval': aligned to math problem key-in time

# there are three periods this code is set up to look at: periods aligned to recall, the entire retrieval period, and the encoding period
recall_type_switch = 0 # how do we select recalls?? Numbers 0:3
# 0: Original analysis taking only recalls without a recall in 2 s IRI before them
# 1: Take these same recalls, but keep only those WITH a recall within 2 s after they occur
# 2: test condition where we look at second recalls within IRI ONLY
# 3: ISOLATED only!
# 4: only first recall of every retrieval period
# 5: take only those recalls that come second in retrieval period within 2 s of first retrieval
# 6: take only NOT first recall of every retrieval period
# 7: take only NOT first recall AND ISOLATED trials (this should REALLY maximize SWR bump)
# 10: same as 0 but with no IRI (mostly just to see number of recalls)

selected_region = HPC_labels #HPC_labels # ENT_labels+HPC_labels

remove_soz_ictal = 0 # 0 for nothing, 1 for remove SOZ, 2 for keep ONLY SOZ ###

min_ripple_rate = 0.1 # Hz.
max_ripple_rate = 1.5 # Hz.
max_trial_by_trial_correlation = 0.05 # if ripples correlated more than this remove them
max_electrode_by_electrode_correlation = 0.2 #???

filter_type = 'hamming' # see local version below for details 
# butter (Vaz algorithm)
# hamming (Norman algorithm)
# hamming125200 (Norman algorithm meant to capture "true" ripple frequency per Sullivan...Buzsaki 2011
# hamming140250 (Same idea, but higher bands)

# patient info
subs = ['UT159', 'UT162', 'UT165', 'UT167', 'UT178', 'UT185', 'UT191', 'UT195', 'UT199', 'UT238'] # UT194 is unusable
# manually assigning the actual session numbers in the behavioral folder in home/eeg/UTXXX/
session_nums = [ [0,1,2],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1,2],[0,1,2],[0,1] ] # note: FR1 and FR1_scopolamine have diff session numbers so no [0,0]s
# note that scopolamine sessions are in .../behavioral/FR1_scopolamine instead of just .../behavioral/FR1
scopo_indicator = [ [0,1,1],[1,1],[1,1],[0,1],[1,1],[1,1],[1,1],[0,1,1],[0,1,1],[1,1] ]

# recall params
recall_minimum = 2000
IRI = 2000 # inter-ripple interval...remove ripples within this range (keep only first one and remove those after it)
retrieval_whole_time = 30000
# encoding params
encoding_time = 2300 # actual preentation is 1.6 s + 0.75-1.0 s so keep +700 ms so can plot +500 ms
pre_encoding_time = -700 # since minimum ISI is 0.75 s let's only plot the 500 ms before word on with a 200 ms buffer
# these aren't likely to be changed:
desired_sample_rate = 500. # in Hz. This seems like lowerst common denominator recording freq.
eeg_buffer = 300 # buffer to add to either end of IRI when processing eeg #**

soz_keep = [0,1] # 0 are good elecs and 1 are SOZ elecs. Never keep 2 (bad leads) ###
if remove_soz_ictal == 1:
    soz_keep = [0]
elif remove_soz_ictal == 2:
    soz_keep = [1]

### END PARAMS ###


if 'entorhinal' in selected_region and 'left hippocampus' in selected_region:
    region_name = 'HPC_ENT'
elif 'entorhinal' in selected_region:
    region_name = 'ENT'
elif 'left hippocampus' in selected_region:
    region_name = 'HPC'
if selected_period == 'surrounding_recall':
    psth_start = -IRI # only makes sense to look at period <= IRI
    psth_end = IRI # how long to grab data after recall
elif selected_period == 'whole_retrieval':
    psth_start = -IRI # doesn't have to be IRI just 2000 ms is convenient
    psth_end = IRI+retrieval_whole_time
elif selected_period == 'encoding':
    psth_start = pre_encoding_time
    psth_end = encoding_time
elif (selected_period == 'math') | (selected_period == 'math_retrieval'): #$$
    psth_start = -2000 # just use 2000 since math problems are actually like 5 s apart typically
    psth_end = 2000   

ripple_array = []; fr_array = []; 
trial_nums = []; 
session_ct = 0; channel_ct = 0; total_channel_ct = 0
HPC_names = []; sub_names = []; sub_sess_names = []
electrodes_per_session = []
total_lists = 0; total_recalls = 0; kept_recalls = 0
align_adjust = 0
ent_elec_ct = []; sd_regions = []; not_sd_regions = []
ripple_ied_accum_ct = []
time_add_save = [];             
encoded_word_key_array = []
list_num_key = []

list_recall_num_array = []; rectime_array = []; recall_before_intrusion_array = [] # new ones added 2020-11-24
serialpos_array = [] # used to be encoding info but commandeered for surrounding_recalls ~~~
recall_position_array = []; recall_index_array = []

trial_by_trial_correlation = []; elec_by_elec_correlation = []
elec_ripple_rate_array = []

channel_coords = []; electrode_labels = []; channel_nums = []


for i_sub,sub in enumerate(subs):
    
    for i_session,session in enumerate(session_nums[i_sub]): # sessions numbers were manually added into a list of lists above
        
        
        ### First get the recall and word events ###
        
        if scopo_indicator[i_sub][i_session] == 1:
            experiment = 'FR1_scopolamine'
        else:
            experiment = 'FR1'
        evs = pd.DataFrame.from_records( read_single_matlab_matrix_as_numpy_structured_array(
            '/data/eeg/'+sub+'/behavioral/'+experiment+'/session_'+str(session)+'/events.mat', 'events') )   
        
        # UTSW data annoyingly uses 'item' instead of 'item_name'
        evs = evs.rename(columns={"item": "item_name"})
            
        # 1) change evs.recalled to False 2) comment out nonrepeat_indicator>0 for good_recalls = (or it removes all the -1s) 
        # 3) set getOutputPositions to [] 4) change save name at bottom to '_intrusions.p'
        evs_free_recall = evs[(evs.type=='REC_WORD') & (evs.intrusion==0)] # recalled word AND correct (from this list...False for instrusions).        
        word_evs = evs[evs['type']=='WORD'] # get words 
        
        # select which recalls??         
        [recall_selection_name,selected_recalls_idxs] = selectRecallType(recall_type_switch,evs_free_recall,IRI,recall_minimum)
      
        _,nonrepeat_indicator = removeRepeatedRecalls(evs_free_recall,word_evs) # remove free recalls that happened more than once
        
        # let's make sure remaining recalls are not repeated, have eeg, and are not from non-practice lists
        good_recalls = selected_recalls_idxs & np.array(evs_free_recall.eegoffset>-1) & np.array(evs_free_recall.list > -1) & (nonrepeat_indicator>0)
        evs_free_recall = evs_free_recall[good_recalls]

        # get temp key of recalls that lead to intrusions ~~~
        pre_intrusion_recalls = getRecallsBeforeIntrusions(evs,evs_free_recall)
        
        if len(evs_free_recall)==0: #...and that any are left!
            continue

        # get output position in each list for this session's recalls
        session_corrected_list_ops = getOutputPositions(evs,evs_free_recall)
        
        
        ### Get UTSW localizations ###
        
        contacts = pd.DataFrame.from_records(np.recfromtxt('/data/eeg/'+sub+'/eeg.noreref/jacksheet.txt', encoding='utf-8'))
        contacts = contacts.rename(columns={'f0':'contact', 'f1':'label'})        
        
        with open('/data/eeg/'+sub+'/docs/'+sub+'_depth_el_info.txt', 'r') as f:
            lines = f.readlines()
        lines = [line.rstrip('\n') for line in lines]

        depth_info = pd.DataFrame(columns=['label', 'description'])
        for line in lines:
            split = line.split()
            try:
                depth_info = depth_info.append({'label':split[1], 'description':' '.join(split[2:])}, ignore_index=True)
            except IndexError:
                continue
        contacts = depth_info.merge(contacts, how='inner') # note it's contacts and not pairs

        
        ###  Load eeg ###
        
        if selected_period == 'surrounding_recall':
            total_recalls = total_recalls + len(evs_free_recall) # get total recalls from lists
            total_lists = total_lists + len(evs[evs.type=='WORD'].list.unique()) # get total lists
            kept_recalls = kept_recalls + len(evs_free_recall)
            eeg_events = evs_free_recall
            
#             # fix EEG offset due to Unity implementation error
#             eeg_events = correctEEGoffset(sub,session,exp,reader,eeg_events) # I dn't think UTSW has this error

        # We want to get the base file name in a clean/systematic way, probably by globbing on the experiment_session name 
        root = '/data/eeg'
        paths = glob(os.path.join(root, sub, 'eeg.noreref', sub+'_'+experiment+'_'+str(session)+'*'), recursive=True)
        file_base = os.path.basename(paths[0]).split('.')[0]
        base_name = os.path.join(os.path.dirname(paths[0]), file_base)
        
        eeg_events['eegfile'] = base_name

        eeg = EEGReader(eeg_events.to_records(),
                        channels= contacts.contact.values[contacts.description.str.contains('hippocampus')],
                        start_time=(psth_start-eeg_buffer+align_adjust)/1000,
                        end_time=(psth_end+eeg_buffer+align_adjust)/1000).read()
        eeg.shape
        
        import ipdb; ipdb.set_trace()

In [24]:
sum(good_recalls)
len(good_recalls)

39

116

### Grab raw events and montage

One very helpful function for reading matlab events!!

### Localization info
Only one thing left: grab the localization info. Looks like it's not so consistently formatted (would be better as a .csv) so maybe necessary to parse line by line

Now we have an updated contacts df with anatomic locations from the UTSW localizations!

## Read EEG
Next, use the base file name in eeg.noreref to read the binary channel data

In [17]:
# We want to get the base file name in a clean/systematic way, probably by globbing on the experiment_session name 

root = '/data/eeg'
experiment = 'FR1_scopolamine'
session = 1
paths = glob(os.path.join(root, sub, 'eeg.noreref', sub+'_'+experiment+'_'+str(session)+'*'), recursive=True)
file_base = os.path.basename(paths[0]).split('.')[0]
base_name = os.path.join(os.path.dirname(paths[0]), file_base)
base_name

events['eegfile'] = base_name
events = events[events.type=='WORD']
events[0:5]

eeg = EEGReader(events.to_records(),
                channels= contacts.contact.values[contacts.description.str.contains('hippocampus')],
                start_time=0, end_time=0.3).read()
eeg.shape

'/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolamine_1_02Aug19_1109'

,subject,session,list,serialpos,type,item,itemno,recalled,mstime,msoffset,rectime,intrusion,isStim,expVersion,stimLoc,stimAmp,stimAnode,stimCathode,stimList,eegfile,eegoffset
24,UT159,1,1,1,WORD,STOVE,256,0,1.564763e+12,1,-999,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,261401
25,UT159,1,1,2,WORD,CALF,41,1,1.564763e+12,1,3850,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,263802
26,UT159,1,1,3,WORD,MUG,162,0,1.564763e+12,1,-999,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,266185
27,UT159,1,1,4,WORD,APE,2,1,1.564763e+12,1,968,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,268802
28,UT159,1,1,5,WORD,FOX,104,1,1.564763e+12,1,1435,-999,0,v_1.05,X,NaN,NaN,NaN,0,/data/eeg/UT159/eeg.noreref/UT159_FR1_scopolam...,271219


(8, 300, 300)

In [18]:
from ptsa.data.filters import MorletWaveletFilter
test = MorletWaveletFilter(eeg, [3., 8.], output='power').filter()
test.shape

CPP total time wavelet loop:  0.19030332565307617


(2, 8, 300, 300)